This is a series of kernels for the NFL Punt Analytics competition:
1. [Game Mechanics](https://www.kaggle.com/argentium/nfl-punt-game-mechanics)
2. [Collision Pairs](https://www.kaggle.com/argentium/nfl-punt-collision-pairs)
3. [Group Dynamics](https://www.kaggle.com/argentium/nfl-punt-group-dynamics)

This is the first part of the series.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import seaborn as sns

import scipy
import math
from matplotlib import pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
df_play_info = pd.read_csv('../input/play_information.csv')
df_video_review = pd.read_csv('../input/video_review.csv')
df_punt_role = pd.read_csv('../input/play_player_role_data.csv')

In [ ]:
# Graph
# Copied from: https://stackoverflow.com/questions/51417483/mean-median-mode-lines-showing-only-in-last-graph-in-seaborn/51417635
def graph_distribution(column):
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.2, 1)})
    mean = column.mean()
    median = column.median()
    mode = column.mode().get_values()[0]

    sns.boxplot(column, ax=ax_box)
    ax_box.axvline(mean, color='r', linestyle='--')
    ax_box.axvline(median, color='g', linestyle='-')
    ax_box.axvline(mode, color='b', linestyle='-')

    sns.distplot(column, ax=ax_hist)
    ax_hist.axvline(mean, color='r', linestyle='--')
    ax_hist.axvline(median, color='g', linestyle='-')
    ax_hist.axvline(mode, color='b', linestyle='-')

    plt.legend({'Mean':mean,'Median':median,'Mode':mode})

    ax_box.set(xlabel='')
    plt.show()

# I. Play Mechanics:

The main objective of the American football is to score as many points as they can by bringing the ball into their respective touchdown zone. For each play, however, the sub-objectives vary according to the role of the team.

There are two possible team roles:
1. Offensive
2. Defensive

In general, the offensive team aims to score a goal by carrying the ball towards the touchdown zone. On the other hand, the defensive team must prevent the offensive team from scoring by tackling the ball carrier.

### What is a Punt Play?

The Punt Play is a specialized play often performed when the offensive team risk losing possession of the ball. The main objective of the offensive team is to kick the ball as far back from the opponent's goal line. To put it simply, the Punt Coverage team might lose ball possession so they want to make it harder for the opponent to score a touchdown by putting more distance from their goal line.

In response the defensive team (Punt Return) attempts to catch the ball to gain possession and move it back as close to their goal line or even score a touchdown if possible.

In a way, Punt Play is special because both teams transition from different roles within a single play. When the Punt Return team (defensive) gains possession of the ball, they turn into an offensive mode while the Punt Coverage team (originally the offensive team) changes into a defensive role.

### Where do the they punt? (How far from their own goal line do the offensive team punt?)

In [ ]:
df_yardline = df_play_info['YardLine'].str.split(" ", n = 1, expand = True)
df_play_info['yard_team'] = df_yardline[0]
df_play_info['yard_number'] = df_yardline[1].astype(float)

# Process Team Sides
df_home_visit = df_play_info['Home_Team_Visit_Team'].str.split("-", n = 1, expand = True)
df_play_info['home'] = df_home_visit[0]
df_play_info['visit'] = df_home_visit[1]

# Convert to coordinate system, origin at goal line
def convert_yardage(row):
    actual_yards = row['yard_number']
    if row['yard_team'] == row['home']:
        return actual_yards
    else:
        return 100 - actual_yards

# Convert to goal line distance
def convert_goal_distance(row):
    if row.loc[('Poss_Team')] == row.loc[('home')]:
        return row.loc[('Scrimmage_Line')]
    else:
        return 100 - row.loc[('Scrimmage_Line')]

df_play_info['Scrimmage_Line'] = df_play_info.apply(lambda row: convert_yardage(row), axis=1)
df_play_info['Goal_Line'] = df_play_info.apply(lambda row: convert_goal_distance(row), axis=1)

# Display Results
graph_distribution(df_play_info['Goal_Line'])
df_play_info['Goal_Line'].describe()

The teams tend to punt when they are midway in their own territory. This means they really want to push the scrimmage line away from their territory.

Sidenote:

The downs and yards needed may be another factor for selecting the punt goal distance. Since I don't have any data on how many downs and yards were needed when these punts were made, I would just refer to the New York Times article:
[4th Down: When to Go for It and Why](https://www.nytimes.com/2014/09/05/upshot/4th-down-when-to-go-for-it-and-why.html).

### What is the most common outcome of Punt Plays?

In [ ]:
results = [ 
           'downed',
           'fair catch', 
           'Touchback',
           'out of bounds'
          ]

def get_result(row):
    for result in results:
        if result in row['PlayDescription']:
            return result

    match = re.search('.* for .*', row['PlayDescription'])
    if match:
        return 'runback'

    return 'others'

df_play_info['Result'] = df_play_info.apply(lambda row: get_result(row), axis=1)

# Graph
ax = sns.countplot(y="Result", 
                   order=df_play_info['Result'].value_counts().index,
                   data=df_play_info)

ax.set_title('Punt Play Outcome Frequency')
df_play_info['Result'].value_counts()

There are several possible outcome of a punt depending on the landing location of the punt:
1. Beyond the Field
    1. End Zone (Touchback)
        - Occurs when the ball reaches the opposite endzone
    2. Out of Bounds
        - Occurs when the lands beyond the field
2. Within the Field
    1. Fair Catch
        - The fair catch is when the PR (from the Return team) decides to catch the ball and declare the ball dead, which means the current play ends with the Return team in possession of the ball. This protects the ball receiver from the opponents.
    2. Downed
        - If the kicking team touches the ball before any return team does, the ball is considered downed.
    3. Runback
        - The ball may stay live if:
            - The return team does not declare a fair catch and catches the ball.
            - The ball is not caught (regardless if a fair catch signal is made), but stays on the field.
        - Under such scenario, the team can make a runback towards the goal line. This allows the return team to regain yards lost due to the punt kick.

### What is the range of distances from the goal line of each result?

In [ ]:
ax = sns.boxplot(x="Goal_Line", y="Result", 
            order=df_play_info.groupby(['Result'])['Goal_Line'].median().index,
            data=df_play_info)
ax.set_title('Goal Distance of Outcomes')

median = df_play_info.groupby(['Result'])['Goal_Line'].median()
median.head()

1. Return (Runback)
    - Returns were made when the goal line is much closer. Perhaps, the sight of a close goal line makes it more tempting for the return team to make a runback. If we compare it to the average distance where punt plays were selected, part of the punt play goal line distance overlap with return goal line distance. This increases the return outcomes compared to other outcomes.
2. Special Cases
    - I will explain this below.
3. Fair Catch
    - Interestingly, the fair catch is made when the yard line is farther from the team's goal line or closer to the opponent's goal line.
        - Perhaps, there is less temptation to return.
        - With less area to cover, there is more chance for the Punt Returner to catch the ball.
4. Downed
    - The median location is similar to the fair catch. A plausible explanation is the smaller coverage area makes it easier to locate the ball.
5. Touchback
    - The touchbacks were made farther away from the team's goal line. This makes sense because it would be easier for the ball to reach the opponent's endzone. Notice that there is no touchback when the goal line is less than 20 yards. This is because the punter's maximum punt distance is 78 yards. No person has ever punted more than 80 yards.

### Is there sufficient variance for every outcome?

In [ ]:
for event in results:
    df_play_info_standard = df_play_info.loc[(df_play_info['Result']!=event) &
                                            (df_play_info['Result']!='others')]
    df_play_info_standard['State'] = 'Standard'

    df_play_info_special = df_play_info[df_play_info['Result']==event]
    df_play_info_special['State'] = 'Event'
    df_concat = pd.concat([df_play_info_standard, df_play_info_special])

    # ANOVA
    mod = ols('Goal_Line ~ State',
                data=df_concat).fit()
    aov_table = sm.stats.anova_lm(mod, typ=2)
    print(str(event) + ":")
    print(aov_table)
    print()

- The out of bounds:
    - The F value is lower than 1. This means there are more variance within the out of bounds group, which means the group does not display a consistent pattern.
    - The PR(>F) is much more than the standard confidence value of 0.05. In other words, the PR(>F) means there's almost half the chance of being wrong in predicting out of bounds. Thus, I considered out of bounds as a completely random event.

Except for the out of bounds event, the rest of the outcomes have sufficient variance with the normal events distribution. This means that the goal distance is an indicator for the following events:
- Return
- Downed
- Fair Catch
- Touchback

### What are the some of the special events?

In [ ]:
special_circumstances = [ 
               'declined', 
               'No Play',
               'pass incomplete',
               'MUFFS catch',
                'RECOVERED',
                'PENALTY'
          ]

def get_others(row):
    for result in special_circumstances:
        if result in row.loc['PlayDescription']:
            return result
    return None

df_play_info['Special_Circumstance'] = df_play_info.apply(lambda row: 
                                                              get_others(row),
                                                              axis=1)

# Graph
ax = sns.countplot(y="Special_Circumstance", 
                   order=df_play_info['Special_Circumstance'].value_counts().index,
                   data=df_play_info)

ax.set_title('Random Outcomes Frequency')
df_play_info['Special_Circumstance'].value_counts()

1. Unintentional: These are understandably random because it comes from human error.
    - Pass incomplete
    - Muffed catch
    - Recovered
2. Penalty Related:
These result to having a punt play technically nullified.
    - No Play
    - Declined

### What is the relationship of outcome percentage with the goal line distance?

In [ ]:
def get_range(row):
    for i in range(20):
        number = 5*(i+1)
        if row['Goal_Line'] < number:
            return number
    return None

df_play_info['Range'] = df_play_info.apply(lambda row: 
                                          get_range(row),
                                          axis=1)

df_play_info['count'] = 1

# Remove the random events
df_play_info_norandom = df_play_info[(df_play_info['Result']!='others')]
df_play_info_range = df_play_info_norandom.groupby(['Range', 'Result']).agg({'count': 'sum'})

# Compute Percantage
df_play_info_range_percent = df_play_info_range.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))
df_play_info_range_percent = df_play_info_range_percent.rename(columns={'count': 'percent'})
df_play_info_range_percent = df_play_info_range_percent.reset_index()

# Statistics
for result in results:
    grouped_range = df_play_info_range_percent[df_play_info_range_percent['Result']==result]

    print(str(result) + ':')
    output = scipy.stats.pearsonr(grouped_range['Range'], 
                        grouped_range['percent'])
    print(output)
    output = scipy.stats.spearmanr(grouped_range['Range'], 
                        grouped_range['percent'])
    print(output)

# Return (Runbacks)
grouped_range = df_play_info_range_percent[df_play_info_range_percent['Result']=='runback']

print(str('runback') + ':')
output = scipy.stats.pearsonr(grouped_range['Range'], 
                    grouped_range['percent'])
print(output)
output = scipy.stats.spearmanr(grouped_range['Range'], 
                    grouped_range['percent'])
print(output)

# Graph
ax = sns.lineplot(x='Range', y='percent',
                     hue='Result',
                     data=df_play_info_range_percent)
ax.set_title('Outcome Probability vs Goal Distance')

To check the general trends, I removed the random events labelled as others. Then, I calculated the percentage of events as rough estimates of the probabilities of the events in the goal distance range.

The positive correlation (percentage increases as the distance increase):
- Fair Catch
- Touchback
- Downed

The negative correlation (percentage decreases as the distance increase):
- Return (Runback)

No correlation:
- Out of bounds

The value of the out of bounds case is very close to zero. This means that the probability of out of bounds is not dependent on the goal distance.

There is also a correlation that I have to consider: the sample size decreases as the goal distance decreases. In simple terms, punts are rarely performed further away from the goal line. Hence, the percentages in the higher ranges (more than 60 yards) may not be reliable.

#### Finding: The probability of punt outcomes relies on the distance from the goal line.

### How far is the ball kicked?

In [ ]:
# Computations
def get_pdistance(row):
    str_punt = '.* punts ([0-9]+) yard'
    distance = re.search(str_punt, row['PlayDescription'], re.IGNORECASE)
    if distance:
        return distance.group(1)
    else:
        return 0

df_play_info['Punts'] = df_play_info.apply(lambda row: get_pdistance(row), axis=1)
df_play_info['Punts'] = df_play_info['Punts'].astype(int)

# Remove null rows
df_play_info_punts = df_play_info.dropna(subset=['Punts'])

# Display Results
graph_distribution(df_play_info_punts['Punts'])
df_play_info_punts['Punts'].describe()

In [ ]:
df_play_info_punts = df_play_info.dropna(subset=['Punts'])
df_play_info_punts_20 = df_play_info_punts[df_play_info_punts['Punts']<20]
print(1-len(df_play_info_punts_20)/len(df_play_info_punts))

The punt kicks often send the ball 45 yards. The maximum punt distance is 78 yards. 96% of the punts are greater than 20 yards.

### Where did it land?

In [ ]:
df_play_info['LandingZone'] = df_play_info['Goal_Line'] + df_play_info['Punts']

# Display Results
graph_distribution(df_play_info['LandingZone'])
df_play_info['LandingZone'].describe()

It landed close to the opponent's endzone. When compared to the initial goal distance, a common expectation is that the initial goal distance mirrors the landing zone. However, this is not exactly the case observed. Notice the 10 yards closest to the opponent's end zone. There is drastic reduction of the number of balls landing in such area. There is a possibility that the ball may have bounced off. However, the other important factor that must be considered is that the punters are not just kicking at random points. Some may have precisely intended for the ball to land at a specific location close to the opponent's end zone, but avoid it from being called a touchback.

While there may be a variety of punt kicks, there is a possible side-effect of moving the touchbacks as described in this article: [Touchback in Kickoffs](https://www.denverpost.com/2016/08/25/nfl-new-touchback-rule-broncos-trying-adjust/).

### How many yards does the Return team gain when runback is made?

In [ ]:
def get_rdistance(row):
    if 'no gain' in row['PlayDescription']:
        return 0

    str_return = '.* for (-*[0-9]+)'
    distance = re.search(str_return, row['PlayDescription'], re.IGNORECASE)
    if distance:
        return int(distance.group(1))
    else:
        return 0

df_play_info['Returns'] = df_play_info.apply(lambda row: get_rdistance(row), axis=1)

# Select cases when there are returns
df_play_info_returns = df_play_info[df_play_info['Result']=='runback']

# Display Results
graph_distribution(df_play_info_returns['Returns'])
df_play_info_returns['Returns'].describe()

For 50% of the return, more than 7 yards were gained. If we consider the yards per play of the top teams, their average is between 6-7 [[Team Yards per Play](https://www.teamrankings.com/nfl/stat/yards-per-play)]. This means, the runback seem beneficial to the Return team. However, the mode is no gain. Thus, the returner must have a good assessment of the current situation of the player.

### Overall, how many yards were the Coverage team able to move the scrimmage line?

In [ ]:
# Computations
df_play_info['PuntGain'] = df_play_info['Punts'] - df_play_info['Returns']

# Graphs
graph_distribution(df_play_info['PuntGain'])
df_play_info['PuntGain'].describe()

Even with some returns, Punt Plays generally send the ball around 42 yards for 50% of the time. 

### Where is the new scrimmage line?

In [ ]:
df_play_info['Next_Goal'] = df_play_info['Goal_Line'] + df_play_info['PuntGain']

# Display Results
graph_distribution(df_play_info['Next_Goal'])
df_play_info['Next_Goal'].describe()

The next scrimmage line is now deep in the enemy's territory. This means that the punt is generally successful.

Now that we know how Punt Plays work, we shall continue with investigating the Punt Play injuries.

# II. Gameplay Injuries

Let us check if there is any pattern on the injury cases according to the game mechanics by reviewing the statistics.

In [ ]:
df_injury = df_video_review.merge(df_play_info, 
                                  left_on=['GameKey', 'PlayID'],
                                 right_on=['GameKey', 'PlayID'],
                                 how='left')

### How many are the Punt Play injuries?

In [ ]:
print(len(df_injury))

There are 37 Punt Play injuries. From the previous statistics, there are 6681 times that Punt formations were used. Thus, injuries occur in 0.5% of the Punt Plays.

### What are the frequent outcome of the injury plays?

In [ ]:
ax = sns.countplot(y="Result", 
                   order=df_injury['Result'].value_counts().index,
                   data=df_injury)
ax.set_title('Frequency Count')
df_play_info['Result'].value_counts()

In the plays with injuries, a return (runback) is often performed.

### What is the riskiest play outcome?

In [ ]:
df_total = df_play_info['Result'].value_counts().reset_index(name='total')
df_injury_total = df_injury['Result'].value_counts().reset_index(name='injured')

df_merged = df_total.merge(df_injury_total, how='right')
df_merged = df_merged.fillna(0)
df_merged['ratio'] = 100*df_merged['injured'] / df_merged['total']
df_merged = df_merged.sort_values(by=['ratio'])

ax = sns.barplot(x='ratio', y='index', 
            data=df_merged)
ax.set_title('Injury Risk per Outcome')

The riskiest play outcome is the return (runback). This is followed by downed. The fair catch is relatively safer than the others.

The return, downed, and fair catch are three different possible outcomes when the ball landed within the field. 

In a punt play, the ball possession transitions from the offensive team to the defensive team. In order to understand injuries related to such outcomes, I decided to categorize the injuries into two phases: (1) Before the punt, and (2) After the punt.

To verify which part of the play is the riskiest, I have to first set-up the data according to the following:
1. Role
2. Team
3. Punt Play Phase

## Role
### Categories

In [ ]:
role_categories = {'G': ['GR', 'GRo', 'GRi',
                        'GL', 'GLo', 'GLi'],
                   'Coverage_Center': ['PRG', 'PLG', 'PRT', 'PLT', 'PRW', 'PLW'],
                  'PP': ['PPR', 'PPRo', 'PPRi',
                         'PPL', 'PPLo', 'PPLi'],
                  'P': ['P'],
                  'PC': ['PC'],
                  'PLS': ['PLS'],
                    'V': ['VR', 'VRo', 'VRi',
                        'VL', 'VLo', 'VLi'],
                  'PD': ['PDR1', 'PDR2', 'PDR3', 'PDR4', 'PDR5', 'PDR6',
                          'PDM',
                         'PDL1', 'PDL2', 'PDL3', 'PDL4', 'PDL5', 'PDL6'],
                  'PL': ['PLR', 'PLR1', 'PLR2', 'PLR3',
                         'PLM', 'PLM1',
                         'PLL', 'PLL1', 'PLL2', 'PLL3', 'PLLi'],
                  'PR': ['PR'],
                  'PFB': ['PFB']
                  }

def set_role_category(role):
    for category in role_categories.keys():
        if str(role) in role_categories[category]:
            return str(category)
    return None

df_punt_role['Role_Category'] = df_punt_role.apply(lambda row: set_role_category(row['Role']), 
                                                axis=1)

### Teams

For each team, there is a set of available roles for their respective formation.

The Punt Play formations are the following:
1. Punt Coverage- The team that kicks the ball is the original offensive team.
2. Punt Return- The team that catches the ball is the original defensive team.

With the player's role, we could identify his team.

In [ ]:
team_positions = {'Return': 
                  ['VR', 'VRo', 'VRi', 
                   'PDR1', 'PDR2', 'PDR3', 'PDR4', 'PDR5', 'PDR6',
                   'PLR', 'PLR1', 'PLR2', 'PLR3',
                   'PR', 'PFB', 'PDM', 'VL', 'VLo', 'VLi',
                   'PDL1', 'PDL2', 'PDL3', 'PDL4', 'PDL5', 'PDL6',
                   'PLL', 'PLL1', 'PLL2', 'PLL3', 'PLLi'],
     'Coverage': ['GR', 'GRo', 'GRi',
                'PRG', 'PRT', 'PRW',
                'PPR', 'PPRo', 'PPRi', 'P', 'PC', 'PLS',
                  'GL', 'GLo', 'GLi',
               'PLW', 'PLT', 'PLG',
               'PPL', 'PPLo', 'PPLi']}

# Add the corresponding side of their role
def set_team(role):
    for team in team_positions.keys():
        if str(role) in team_positions[team]:
            return str(team)
    return None

df_punt_role['Team'] = df_punt_role.apply(lambda row: set_team(row['Role']), 
                                                axis=1)

### Merge Role Information with Injury Data

In [ ]:
# Punt Roles
# Convert to int data type
df_injury['Primary_Partner_GSISID'] = df_injury.apply(lambda row: 
                                                                  row['Primary_Partner_GSISID'] 
                                                                  if (row['Primary_Partner_GSISID'] != 'Unclear')
                                                                 else 0,
                                                                 axis=1)
df_injury['Primary_Partner_GSISID'] = df_injury['Primary_Partner_GSISID'].fillna(0)
df_injury['Primary_Partner_GSISID'] = df_injury['Primary_Partner_GSISID'].astype(int)

# Identify roles for player and partner
df_injury = df_injury.merge(df_punt_role, 
                                  left_on=['GameKey', 'PlayID', 'GSISID'],
                                 right_on=['GameKey', 'PlayID', 'GSISID'],
                                 how='left')

df_injury = df_injury.merge(df_punt_role, 
                                 suffixes=('', '_Partner'),
                                  left_on=['GameKey', 'PlayID', 'Primary_Partner_GSISID'],
                                 right_on=['GameKey', 'PlayID', 'GSISID'],
                                 how='left')
df_injury = df_injury.drop(['GSISID_Partner', 'Season_Year_Partner'], axis=1)

### How many null values are in the data?

In [ ]:
df_injury.isnull().sum()

### What are the conditions of the unknown role partners?

In [ ]:
df_injury_null = df_injury[pd.isnull(df_injury['Role_Partner'])]
df_injury_null.head()

The impact type is either helmet-to-ground to those without a partner activity. This means there is no partner at all.
I cleaned the data by replacing some null values.

In [ ]:
df_injury = df_injury.fillna({'Friendly_Fire': 'Ground',
                             'Role_Partner': 'Unknown'})

## Game Phases

For us to understand the dynamics between the Coverage and Return teams, we need to identify the goals of the teams for each specific turning points in the game.

There are two main goals for each team in a Punt Play depending on the Phase:

1. Ball on Punt Coverage
    1. Punt Return team attempts to tackle the Punter
    2. Punt Coverage team blocks the Punt Return Team to defend their Punter

2. Ball on Punt Return
    1. Punt Return catches the ball and must avoid getting tackled
    2. Punt Coverage team attempts to tackle the Punt Return

For each goal, we identify the game phase according to the following activities according to the Punt team.
1. Before the Kick:
    1. Coverage = Tackled or Blocking
    2. Return = Tackling or Blocked
2. After the Kick:
    1. Coverage = Tackling or Blocked
    2. Return = Tackled or Blocking

In [ ]:
def get_goal(activity):
    if (activity == 'Blocking') or (activity == 'Tackled'):
        return 'Offensive'
    else:
        return 'Defensive'

# Add the corresponding side of their role
def set_phase(row):
    goal = get_goal(row['Player_Activity_Derived'])
    if row['Team'] == 'Coverage':
        if goal == 'Offensive':
            return 1
        else:
            return 2
    else: # Return Team
        if goal == 'Offensive':
            return 2
        else:
            return 1

# Merge with df_punt_role
df_injury['Phase'] = df_injury.apply(lambda row: 
                                                set_phase(row), 
                                                axis=1)

### What phase has more injuries?

In [ ]:
ax = sns.countplot(x="Phase", data=df_injury)
ax.set_title('Phase Injury Frequency')

The second phase has an alarmingly higher number of injuries than the first phase. Nevertheless, we have to study each one.

## A. Before the kick:

### How many injuries were there?

In [ ]:
df_phase1 = df_injury[df_injury['Phase']==1]
print(len(df_phase1))

### What are the common outcomes of the injuries before the kick?

In [ ]:
# df_phase1['Result'].value_counts()
ax = sns.countplot(x="Result", data=df_phase1)
ax.set_title('Before the Kick: Outcome Frequency')

### What is the most common activity leading to an injury before the kick?

In [ ]:
df_cross_injured = pd.crosstab(df_phase1['Player_Activity_Derived'], df_phase1['Primary_Partner_Activity_Derived'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.1g')
ax.set_title('Before the Kick: Activity Pairs')

Most of the injuries were from blocking-blocked activities, but there is one tackled-tackling injury. 

### What are the circumstances of the single tackled injury?

In [ ]:
df_phase1_tackled = df_phase1[df_phase1['Player_Activity_Derived'] == 'Tackled']
df_phase1_tackled.head()

Upon video review, however, the injury is a rare spur-of-the-moment case of the Punter running with the ball rather than kicking it.
The VLo was able to tackle the Punter because the V position is away from the crowd.

### For the blocking injuries, are the common pair of roles?

In [ ]:
df_phase1_blocking = df_phase1[df_phase1['Player_Activity_Derived'] == 'Blocking']

# Graph
df_cross_injured = pd.crosstab(df_phase1_blocking['Role'], 
                               df_phase1_blocking['Role_Partner'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('Before the Kick: Blocked Role Pairs')

Note that the left side of the Coverage team is the right side of the Return team. From the roles, the injuries can be identified from the same side of the field, but from opposing teams. Upon video review, some wingmen were injured when pushed at an angle by a front-row opponent right before them.


### What is the most common impact type of the blocked injuries?

In [ ]:
ax = sns.countplot(y="Primary_Impact_Type", 
                   order = df_phase1_blocking['Primary_Impact_Type'].value_counts().index,
                   data=df_phase1_blocking)
ax.set_title('Before the Kick: Blocking Impact Type Frequency')

Most of the blocking injuries are helmet-to-helmet. With video review, it can be seen that the players were wrestling each other head-on when the injuries occured.

#### Finding 1: The Coverage wing positions have a dangerous angle.

While there is some risk in the Coverage formation, however, a larger part of the injuries come from the second phase.

## B. After the Kick:

Let us refer back to the Phase 2 goals of each team:
    1. Coverage = Tackling or Blocked
    2. Return = Tackled or Blocking

### How many injuries occured after the kick?

In [ ]:
df_phase2 = df_injury[df_injury['Phase']==2]
print(len(df_phase2))

### After the kick, what is the most common injured role?

In [ ]:
ax = sns.countplot(y="Role", 
                   order = df_phase2['Role'].value_counts().index,
                   data=df_phase2)
ax.set_title('After the Kick: Injury Frequency')

The PR has the highest frequency of injury.

### What is the ratio of role injuries out of the times the role is used?

In [ ]:
df_total = df_punt_role['Role'].value_counts().reset_index(name='total')
df_injury_total = df_phase2['Role'].value_counts().reset_index(name='injured')

# df_total.head()
df_merged = df_total.merge(df_injury_total, how='right')
df_merged = df_merged.fillna(0)
df_merged['ratio'] = 100*df_merged['injured'] / df_merged['total']
df_merged = df_merged.sort_values(by=['ratio'], ascending=False)

ax = sns.barplot(x='ratio', y='index', 
            data=df_merged)
ax.set_title('After the Kick: Role Injury Ratio')
df_merged.head()

The PR ranks second in terms of percentage injury. However, the top role (PFB) is only injured once out of the few times the role was used.

### After the kick, what is the most frequent partner role?

In [ ]:
ax = sns.countplot(y="Role_Partner", 
                   order = df_phase2['Role_Partner'].value_counts().index,
                   data=df_phase2)
ax.set_title('After the Kick: Partner Frequency')

Again, the PR is the most common role, but as a collision partner.

### What is the ratio of partner role involvement out of the times the role is used?

In [ ]:
df_total = df_punt_role['Role'].value_counts().reset_index(name='total')
df_injury_total = df_phase2['Role_Partner'].value_counts().reset_index(name='injured')

# df_total.head()
df_merged = df_total.merge(df_injury_total, how='right')
df_merged = df_merged.fillna(0)
df_merged['ratio'] = 100*df_merged['injured'] / df_merged['total']
df_merged = df_merged.sort_values(by=['ratio'], ascending=False)

ax = sns.barplot(x='ratio', y='index', 
            data=df_merged)
ax.set_title('After the Kick: Role Injury Ratio')
df_merged.head()

The PR still ranks as the 2nd highest in terms ratio. The top rank (PLL1) only has one injury out of the very few times the role was used.

### What are the common pairs of activities involving the PR?

In [ ]:
df_phase2_pr = df_phase2[(df_phase2['Role']=='PR') | (df_phase2['Role_Partner']=='PR')]

# Graph
df_cross_injured = pd.crosstab(df_phase2_pr['Player_Activity_Derived'], 
                               df_phase2_pr['Primary_Partner_Activity_Derived'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.1g')
ax.set_title('After the Kick: PR-related Activity Pairs')

The PR-related injuries is a Tackling-Tackled pair of activity. This is expected after the kick because the PR's main responsibility is catching the ball and running with it. However, the tackling player gets injured more often than the Punt Returner himself.

### What are the roles often injured when involved with the PR?
For the tackled-tackling pairs, we expect the PR to be the target because he is tasked with catching the kicked ball.

In [ ]:
df_phase2_pr = df_phase2[(df_phase2['Role']=='PR') | (df_phase2['Role_Partner']=='PR')]

# Graph
df_cross_injured = pd.crosstab(df_phase2_pr['Role'], df_phase2_pr['Role_Partner'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.1g')
ax.set_title('After the Kick: PR-related Partner Roles')

The PRG has frequently targeted the PR during the injuries. The difference between other roles is not much.
- There are 11 roles in a team. 
- 8 identified roles implies that almost every Coverage team role got involved with an injury while targeting the PR.

We are starting to see a major game flaw in this tackling-tackled activity pairs.
1. The tackled activity risk is not evenly distributed amongst the Return team.
2. The tackling activity risk is evenly distributed amongst the Coverage team.

### For the PR-related injuries, how does the tackled injury differ from the tackling injury?

In [ ]:
df_cross_injured = pd.crosstab(df_phase2_pr['Player_Activity_Derived'], df_phase2_pr['Primary_Impact_Type'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Activity Impact Types\n(PR-related Injuries)')

There are higher number of helmet-to-body collisions of the tackling player. When combined with the movement data, the PR is often moving in the same direction along the gridline. The tackling player was forced to lunge forward towards the PR because the he was about to be left behind. Moreover, there are several instances of more than one player tackling the PR at the same time.

On the other hand, the tackled injuries of the PR had just one additional helmet-to-helmet injuries compared to the helmet-to-body injury.

#### Finding 2: The single targeting of the Punt Returner is risky for both the PR and tackling player.

### What role category has the most frequent injury?

In [ ]:
ax = sns.countplot(y='Role_Category', 
              order = df_phase2['Role_Category'].value_counts().index,
            data=df_injury)
ax.set_title('After the Kick: Role Category Frequency')

The top three roles are from the Coverage team.
### What is the riskiest role category? (What is the ratio of injuries out of the times the role is used?)

In [ ]:
df_total = df_punt_role['Role_Category'].value_counts().reset_index(name='total')
df_injury_total = df_phase2['Role_Category'].value_counts().reset_index(name='injured')

# df_total.head()
df_merged = df_total.merge(df_injury_total, how='right')
df_merged = df_merged.fillna(0)
df_merged['ratio'] = 100*df_merged['injured'] / df_merged['total']
df_merged = df_merged.sort_values(by=['ratio'], ascending=False)

ax = sns.barplot(x='ratio', y='index', 
            data=df_merged)
ax.set_title('After the Kick: Role Injury Ratio')
df_merged.head()

Interestingly, the most riskiest roles come from the side of the Coverage team. This means they are more likely to get injured.

Note: The PFB is an exception because it only has 1 injury out of the few times it is played.

### What are the most common partner roles in the injuries?

In [ ]:
ax = sns.countplot(y='Role_Category_Partner', 
              order = df_injury['Role_Category_Partner'].value_counts().index,
            data=df_injury)
ax.set_title('After the Kick: Role Partner Frequency')

The Coverage team roles are the most common partners. Although the PR is only a single role out of the grouped role categories, it tops the list as the second most common partner role.

### What is the most dangerous partner role? (What is the ratio of partner role involvement out of the times the role is used)?

In [ ]:
df_total = df_punt_role['Role_Category'].value_counts().reset_index(name='total')
df_injury_total = df_phase2['Role_Category_Partner'].value_counts().reset_index(name='injured')

# df_total.head()
df_merged = df_total.merge(df_injury_total, how='right')
df_merged = df_merged.fillna(0)
df_merged['ratio'] = 100*df_merged['injured'] / df_merged['total']
df_merged = df_merged.sort_values(by=['ratio'], ascending=False)

ax = sns.barplot(x='ratio', y='index', 
            data=df_merged)
ax.set_title('After the Kick: Role Partner Ratio')

df_merged.head()

Since the PR is the only single role among the grouped categories and it was also the most frequent partner role, it has a bigger percentage of injuries compared to the others.

### What are the most common impact type per role category?

In [ ]:
# Graph
df_cross_injured = pd.crosstab(df_phase2['Role_Category'], 
                               df_phase2['Primary_Impact_Type'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Role Category and Impact Type')

1. Coverage center positions
    - Most suffered from helmet-to-body injuries.
    - Helmet-to-helmet comes in second.
    - The Coverage team is the only one to suffer helmet-to-ground impacts.
2. Gunner
    - 4/5 of injuries are helmet-to-helmet impacts

### What are the activity pairs when the Gunner is injured?

In [ ]:
df_phase2_gunner_player = df_phase2[(df_phase2['Role_Category']=='G')]

# Graph
df_cross_injured = pd.crosstab(df_phase2_gunner_player['Player_Activity_Derived'], 
                               df_phase2_gunner_player['Primary_Partner_Activity_Derived'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Activity Pairs\n(Gunner is Injured)')

The activities of the Gunner is equally distributed to Blocked and Tackling. However, there is one outlier with both players Tackling.

### What is the circumstances of the same activity injury when the gunner is injured?

In [ ]:
df_phase2_gunner = df_phase2[(df_phase2['Role_Category']=='G') |
                            (df_phase2['Role_Category_Partner']=='G')]

df_phase2_gunner_tackling = df_phase2_gunner[(df_phase2_gunner['Player_Activity_Derived']=='Tackling') &
                                            (df_phase2_gunner['Primary_Partner_Activity_Derived']=='Tackling')]

df_phase2_gunner_tackling.head()

Both the injured and partner role are Gunners. The injured role is the left gunner while the partner role is the right gunner.

### What are the common pair of roles involved in Gunner-related injuries?

In [ ]:
df_phase2_gunner = df_phase2[(df_phase2['Role_Category']=='G') |
                            (df_phase2['Role_Category_Partner']=='G')]

# Graph
df_cross_injured = pd.crosstab(df_phase2_gunner['Role_Category'], 
                               df_phase2_gunner['Role_Category_Partner'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Gunner Pairs')

While the common expected pair is the Gunner and PR, the Gunner also collided with other players. Interestingly, the Gunner collided with a fellow Gunner even if there is usually just 2 gunners in a punt play.

While the PR is the expected target, the injured roles are uniformly distributed to the 4 injured players. The roles injured by the gunner roles are equally from both teams (2 for Return team: PL, PR; 2 for Coverage team: G, Coverage Center).

Compared to the PR where all other players were targeting him, the Gunner is not particularly targeted, but he is targeting the PR.

### What are the activity pairs when the Gunner injured someone?

In [ ]:
df_phase2_gunner_partner = df_phase2[(df_phase2['Role_Category_Partner']=='G')]

# Graph
df_cross_injured = pd.crosstab(df_phase2_gunner_partner['Player_Activity_Derived'], 
                               df_phase2_gunner_partner['Primary_Partner_Activity_Derived'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Activity Pairs\n(Gunner is Partner Role)')

- There is an equal distribution of activities to all players.
- On the side of the Gunner, his activities are equally distributed to Blocked and Tackling. 
- Interestingly, there are same activity injuries. This implies friendly-fire injuries.

### What are the impact types when the Gunner injured someone?

In [ ]:
ax = sns.countplot(y='Primary_Impact_Type', 
              order = df_phase2_gunner_partner['Primary_Impact_Type'].value_counts().index,
            data=df_phase2_gunner_partner)
ax.set_title('After the Kick: Impact Types\n(Gunner is Partner Role)')

df_phase2_gunner_partner['Primary_Impact_Type'].value_counts()

The impact types are evenly distributed to helmet-to-helmet and helmet-to-body collisions.

### How many friendly-fire collisions involve the gunner?

In [ ]:
ax = sns.countplot(y='Friendly_Fire', 
              order = df_phase2_gunner['Friendly_Fire'].value_counts().index,
            data=df_phase2_gunner)
ax.set_title('After the Kick: Friendly-Fires\n(Gunner is Involved)')

df_phase2_gunner['Friendly_Fire'].value_counts()

2/8 or a quarter of the gunner collisions are friendly-fires.

#### Finding 3: The Gunner role is another vulnerable role with a lot of helmet-to-helmet injuries and some friendly-fires.

### Without the PR-related injuries, how common are the non-friendly fire injuries?

In [ ]:
df_phase2_nopr = df_phase2[(df_phase2['Role']!='PR') &
                          (df_phase2['Role_Partner']!='PR')]

ax = sns.countplot(y='Friendly_Fire', 
              order = df_phase2_nopr['Friendly_Fire'].value_counts().index,
            data=df_phase2_nopr)
ax.set_title('After the Kick: Opponent Injuries')

df_phase2_nopr['Friendly_Fire'].value_counts()

The most common injuries comes from opponents. This is expected because the teams are against each other. However, there are also friendly-fires and no partner injuries.

### Without the PR-related injuries, what is the most common pair of team matchup in the injuries?

In [ ]:
# Graph
df_cross_injured = pd.crosstab(df_phase2_nopr['Team'], 
                               df_phase2_nopr['Team_Partner'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Team vs Team Injuries \n(Without PR Injuries)')

The injuries between coverage vs return, return vs coverage, and coverage vs coverage are uniformly distributed. Interestingly, the friendly-fires of the coverage team is around the same level as the opponent-vs-opponent injuries. It would seem like the coverage team is against itself. On the other hand, the return team has no friendly-fires.

### Without the PR-related injuries, what is the most frequent activity pairs in the non-friendly fire injuries?

In [ ]:
df_phase2_nopr_opponent = df_phase2_nopr[df_phase2_nopr['Friendly_Fire']=='No']

# Graph
df_cross_injured = pd.crosstab(df_phase2_nopr_opponent['Player_Activity_Derived'], 
                               df_phase2_nopr_opponent['Primary_Partner_Activity_Derived'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Opponent Activity Pairs\n(Without PR-related injuries)')

- The blocking and blocked injuries are uniformly distributed.
- There is an outlier of Tackling injury while the partner opponent is Blocking.

#### Finding 4: Without the PR-related injuries, most of the opponent injuries are related to blocks.

### What are the common block impact types per team?

In [ ]:
# Graph
df_cross_injured = pd.crosstab(df_phase2_nopr_opponent['Team'], 
                               df_phase2_nopr_opponent['Primary_Impact_Type'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Team vs Team Injuries \n(Without PR Injuries)')

Most of the coverage team suffered helmet-to-helmet injury types than the return team, which had more helmet-to-body injuries from blocks.

### What are the common block impact types per team without the gunner collisions?

In [ ]:
df_phase2_nopr_opponent_nogunner = df_phase2_nopr_opponent[df_phase2_nopr_opponent['Role_Category']!='G']

# Graph
df_cross_injured = pd.crosstab(df_phase2_nopr_opponent_nogunner['Team'], 
                               df_phase2_nopr_opponent_nogunner['Primary_Impact_Type'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Team vs Team Injuries \n(Without PR or Gunner Injuries)')

Without the gunner collisions, the block injuries are evenly distributed to different impact types for each team.

### Overall, how many injuries does each team have according to opponent match-up?

In [ ]:
# Graph
df_cross_injured = pd.crosstab(df_phase2['Team'], df_phase2['Friendly_Fire'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Team Opponent Injuries\n(Overall)')

Coverage vs Return team injuries are expected because they are opposing each other.
- However, there is a higher amount of injuries from the coverage team because of friendly fires.
- Moreover, friendly fires are only on the Coverage team while the Return team does not have any friendly fire.
- We also note that there are two incidents of the Coverage team hitting the ground.
- In one incident, the partner pair is unclear for the Coverage team.

### What are the common activities of friendly fire?

In [ ]:
df_friendly_fire = df_phase2[(df_phase2['Friendly_Fire']=='Yes')]

# Graph
df_cross_injured = pd.crosstab(df_friendly_fire['Player_Activity_Derived'], df_friendly_fire['Primary_Partner_Activity_Derived'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Friendly-fire Activity Pairs')

The friendly fires primarily consist of same activities for the team. This indicates a failure in the interaction between members of the same team (Coverage team). They both got in the way of each other. From the previous analysis on the Gunner, we found out that the gunner has some friendly-fire injuries.

This means that without the gunner injuries, each activity pair would have a uniform distribution of 1 each except for the Tackling-Blocked pair, which has no injury.

### Without the gunner collisions, what are the friendly-fire activity pairs?

In [ ]:
df_friendly_fire = df_phase2[(df_phase2['Friendly_Fire']=='Yes')]
df_friendly_fire_nogunner = df_friendly_fire[(df_friendly_fire['Role_Category']!='G') &
                                            (df_friendly_fire['Role_Category_Partner']!='G')]

# Graph
df_cross_injured = pd.crosstab(df_friendly_fire_nogunner['Player_Activity_Derived'], 
                               df_friendly_fire_nogunner['Primary_Partner_Activity_Derived'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Friendly-fire Activity Pairs\n(No Gunner Collisions)')

Without Gunner the collisions, most injuries came from both tackling players.

### Without the gunner collisions, what are the friendly-fire role pairs?

In [ ]:
# Graph
df_cross_injured = pd.crosstab(df_friendly_fire_nogunner['Role_Category'], 
                               df_friendly_fire_nogunner['Role_Category_Partner'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Friendly-fire Role Pairs\n(No Gunner Collisions)')

Most of the player pairs came from the coverage center.

### Without the gunner collisions, what are the common impact types of the injuries?

In [ ]:
# Graph
ax = sns.countplot(y="Primary_Impact_Type", 
                   order = df_friendly_fire_nogunner['Primary_Impact_Type'].value_counts().index,
                   data=df_friendly_fire_nogunner)
ax.set_title('After the Kick: Friendly-fire Impact Types\n(No Gunner)')
df_friendly_fire_nogunner['Primary_Impact_Type'].describe()

Most of the friendly fire injuries without the gunner are helmet-to-body collisions.

### What are the circumstances of those without partner?

In [ ]:
df_phase2_ground = df_phase2[df_phase2['Friendly_Fire']=='Ground']

# Graph
df_cross_injured = pd.crosstab(df_phase2_ground['Player_Activity_Derived'],
                               df_phase2_ground['Role'])
df_cross_injured = df_cross_injured.fillna(0)

ax = sns.heatmap(df_cross_injured, annot=True, fmt='.2g')
ax.set_title('After the Kick: Activity and Role Pairs\n(Ground Injuries)')

The curious cases of no partner injuries are primarily helmet-to-ground injuries.
- The PLG was injured when blocked.
- The PLW was injured while tackling.

Upon video review, the circumstances show that:
- The PLG was injured while running with the crowd of his team along with some opponents on the way.
- On the other hand, the PLW was injured while tackling because he missed the target.

In both cases, the players were running fast towards a single target: the PR.

This brings me to review the overall characteristic of the Coverage team.

### What activity after the kick has the most frequent injury?

In [ ]:
ax = sns.countplot(y="Player_Activity_Derived", 
                   order = df_phase2['Player_Activity_Derived'].value_counts().index,
                   data=df_phase2)
ax.set_title('After the Kick: Activity Injury Frequency')
df_phase2['Player_Activity_Derived'].describe()

Tackling is a major cause of injuries after the punt. The second activity, Blocked, implies an injury on the Coverage side. In other words, the top two activities are related to the Coverage side activities.

#### Finding 5: The Coverage team gets more injuries from friendly-fires and tackling attempts.

# III. Findings Summary:

## 1. Coverage Formation Blocking

- The wing positions are a weak spot of the coverage formation.

### How are the injuries related to the team objectives?
1. Coverage
    - Team objective: Enable the Punter to freely kick by blocking the opponent.
2. Return
    - Team objective: Stop the Punt.

In simple terms, the Coverage blocks the Return team from stopping the punt

## 2. PR-Related Injuries
- The PR is the most frequently injured role and partner role in the injuries.
- All tackling-tackled injuries are related to the PR.

### How are the injuries related to the role of a Punt Returner?
A punt returner:
> " specializes in returning punts" -Wikipedia

In other words, the Punt Returner catches the ball and may run with it. This makes him the main target.

## 3. Gunner-Related Injuries
- The Gunner role is the 4th most likely to get injured.
- The Gunner role is 2nd role most likely to injure others.

### How are the injuries related to the role of a Gunner?

A gunner:
> "specializes in running down the sideline very quickly in an attempt to tackle the kick or punt returner" -Wikipedia

This means running fast is within his job description.

## 4. Opponent Block Injuries
- Without the PR-related injuries, most of the opponent injuries are related to blocks.
- The block injuries are uniformly distributed after the kick.

### How are the injuries related to the team objectives?
1. Coverage
    - The Coverage team must tackle the PR to prevent him from gaining yards.
2. Return
    - Hence, the Return team responds by blocking the Coverage team from reaching the PR.
    
In simple terms, blocking is part of the game.

## 5. Self-Inflicted Coverage Team Injuries
- Most of the friendly fire injuries come from the coverage team.
- The most common friendly-fire activity pairs come from similar activities.
- The Coverage team's tackling is the most dangerous activity.
- This is followed by getting blocked.

### How are the injuries related to the individual's objectives?
We review the objectives of the teams.
1. Coverage
    - Team objective: Tackle the PR
    - Personal objective: __Be the one to tackle the PR__
2. Return
    - Team objective: Gain yardage/Protect the PR
    - Personal objective: Block the opponent

If we look at the difference between group objectives and personal objectives, the circumstances of the Coverage team makes them compete against each another. This can be observed in several videos where more than 1 Coverage team member tackled the PR.
On the other hand, the Return team is on a defensive mode, which makes them more careful and aware of each other.

The main issue here is similar to the bottleneck or single door crowd dynamics. In the single door crowd dynamics, the crowd accumulates because there is only one target location. Such situation increases the interaction between the people, which may result to injuries.

Reference:
[Crowd Dynamics](http://www.gkstill.com/CV/PhD/Chapter3.html
![image.png](attachment:image.png)

----
In summary, the team and individual behaviors are driven by the game objectives.

This kernel explains the basics of the game mechanics and its relation to the injuries.
The movement dynamics would better explain how the execution of such behaviors lead to injuries. Since the NGS requires some heavy data processing, I separated it into another kernel.

This is the outline of the next kernel:
1. Phase 1:
    1. Coverage Formation Blocking
2. Phase 2:
    1. Punt Returner Collisions
    2. Gunner Collisions
    3. Block Opposition
    4. Friendly-Fire Collisions (Without the Gunner)